In [5]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [7]:
from accounts import Account

In [8]:
account = Account.get("John Doe")
account

Account(name='john doe', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [9]:
account.buy_shares("AMZN", 3, "Because they recently did mass layoffs.")

'Completed. Latest details:\n{"name": "john doe", "balance": 9265.87468, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 244.70844, "timestamp": "2025-11-02 23:10:45", "rationale": "Because they recently did mass layoffs."}], "portfolio_value_time_series": [["2025-11-02 23:10:45", 9998.53468]], "total_portfolio_value": 9998.53468, "total_profit_loss": -1.4653199999993376}'

In [10]:
account.report()

'{"name": "john doe", "balance": 9265.87468, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 244.70844, "timestamp": "2025-11-02 23:10:45", "rationale": "Because they recently did mass layoffs."}], "portfolio_value_time_series": [["2025-11-02 23:10:45", 9998.53468], ["2025-11-02 23:12:20", 9998.53468]], "total_portfolio_value": 9998.53468, "total_profit_loss": -1.4653199999993376}'

In [11]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 244.70844,
  'timestamp': '2025-11-02 23:10:45',
  'rationale': 'Because they recently did mass layoffs.'}]

In [13]:
# use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='buy_shares', 

In [14]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is John Doe and my account is under the name John Doe. What's my balance and my holdings?"

In [16]:
import os
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
google_api_key = os.getenv('GOOGLE_API_KEY')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

In [24]:
from contextlib import nullcontext as trace

In [ ]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=gemini_model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

OK. Your cash balance is $9265.87 and you have 3 shares of AMZN.


[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: AIzaSyC4***************************eAYg. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
